# Project 04: Language Models

## Checkpoint Due Date (Questions 1-5): Wednesday, May 22 11:59 PM

## Due Date: Tuesday, May 28 11:59 PM

#### Checkpoint Instructions
* The checkpoint requires you to turn in questions 1-5.
* The checkpoint will be graded for approximate correctness: easier than the final tests; harder than the doctests.
* The checkpoint will count toward the extra-credit grade at the end of the course.

# Introduction to Language Models

In this project, you will build *[statistical language models](https://en.wikipedia.org/wiki/Language_model)* using public domain books found on [Project Gutenberg](https://www.gutenberg.org/). Language models attempt to capture the likelihood that a given sequence of words occur in a given "language" (the precise term is "corpus" or "corpora"). Here, "language" is a broad term that, in addition to the normal usage, may mean the language of a particular author or style. Applications of language models are found in areas such as language generation, authorship attribution, auto-complete / word-suggestion, and machine translation.

In this project, you will learn about a simple but effective type of language model (LM) called N-gram models. Given a sentence (that is, a sequence of words $w = w_1\ldots w_n$), you would like to understand the probability that sentence is used: 
$$P(w) = P(w_1,\ldots,w_n)$$
However, sentences are built from words, and the likelihood that a word occurs where it does, depends on the words it follows. This points to using *conditional probability* to understand $P(w)$. That is, we can write:

$$
P(w) = P(w_1,\ldots,w_n) = P(w_1) \cdot P(w_2|w_1) \cdot P(w_3|w_1,w_2) \cdot\ldots\cdot P(w_n|w_1,\ldots,w_{n-1})
$$

**Example:** Consider the sentence $w = $ `when I eat pizza, I wipe off the grease.`

$$
P(\mbox{when}) \cdot P(\mbox{I | when}) \cdot P(\mbox{ eat | when I})\cdot P(\mbox{ pizza | when I eat}) \cdot \ldots \cdot P(\mbox{ grease | when I eat pizza, I wipe off the})
$$

That is, the probability that the sentence occurs is the product of the probability that each subsequent word follows the words that came before. For example, the probability $P(\mbox{ pizza | when I eat})$ is pretty high, as pizza is something that you eat; thus, this term would contribute to this sentence having a higher probability of occurrence.

### N-gram language models

In the description above, the probability that any word in the sentence (e.g. 'grease') depends on *every* word that preceded it (e.g. 'when'). However, it's often the case the likelihood a word appears in a sentence is influenced more by *nearby* words. 
* N-gram language models capture this concept that only nearby words matter; they assume the probability a word occurs only depends on the previous $(N−1)$ words. 
* That is, an N-gram model says: $P(w_n|w_1,\ldots,w_{n-1}) = P(w_n|w_{n-(N-1)},\ldots,w_{n-1})$.

**Example: trigram model**

Consider the sentence $w = $ `when I eat pizza, I wipe off the grease.`

$$
P(\mbox{when}) \cdot P(\mbox{I | when}) \cdot P(\mbox{ eat | when I})\cdot P(\mbox{ pizza | I eat}) \cdot \ldots \cdot P(\mbox{ the | wipe off}) \cdot P(\mbox{ grease | off the})
$$
In this example, we see the trigram model doesn't consider the beginning of the sentence when considering the likelihood of the end word 'grease'.

**Definition:** Given the descriptions and definitions above, we define the **N-grams of a text** to be the list of sliding windows of length N.

For Example, the trigrams of the sentence $w$ in the example above are:
```
[('when', 'I', 'eat'), ('I', 'eat', 'pizza'), ... , ('wipe', 'off', 'the'), ('off', 'the', 'grease')]
```

### Computing an N-gram language model

N-gram language models are a collection of conditional probabilities for sequences of words of length $N$. For example, in a trigram model, for every 3-word sequence $w_1, w_2, w_3$, you need to compute:
$$P(w_1,w_2,w_3) = P(w_3 | w_1, w_2) = \frac{C(w_1, w_2, w_3)}{C(w_1, w_2)}$$

Where $C(w_1, w_2, w_3)$ is the number of occurrences of the trigram-sequence $w_1, w_2, w_3$ in the dataset and $C(w_1, w_2)$ is the number of occurrences of the bigram-sequence  $w_1, w_2$ in the dataset.

For a general N-gram model, the conditional probabilities are computed by dividing the counts of N-grams by the (N-1)-grams they follow.


### Tokenizing corpora

Computing the probabilities of a language model from a book requires breaking up the text of book into sequences of words. This process is called *tokenization*. In reality though, the sequences are not made up entirely of words, but rather more general terms called *tokens*. In this project tokens will include not only whole words, but also punctuation and other terms. Below are a few examples of other types of tokens:

* Punctuation like `,.;'`. For example, the period makes sense as a token, as certain words tend to end sentences (i.e. come before a `.` in an N-gram) and begin sentences (i.e. come after a `.` in an N-gram).
* We have special 'START' and 'END' tokens that begin and end every word sequence (in our case, paragraphs of words in a given book). These make sense as tokens, as certain words may tend to begin and end paragraphs. 

It is useful for the tokens used to represent START and END to be single characters that can never be found in the text of the book you use to create our language model. Thus, you will use two ascii hidden "[control characters](https://en.wikipedia.org/wiki/C0_and_C1_control_codes#C0_(ASCII_and_derivatives))":
* For START, you will use the character `\x02` (used for 'beginning of text')
* For END, you will use the character `\x03` (used for 'end of text')


### A note on checking your work for correctness

Building a statistical model involves stringing together difficult-to-follows steps such as data-cleaning and implementing derived mathematical formulas. A fact of life in this line of work is that it can be difficult to assess the correctness of your work. 

* The Doctests will include checks on the inputs/outputs of your methods.
* Doctests will include models on which you can check your work 'by hand'.
* Additionally, **you should run your functions on *real* books and assess the reasonableness of your result** as well. Merely passing the doctests is never sufficient to guarantee correctness of your code.

# Project Outline

Below is an outline of the the project. If there are terms you don't understand in following descriptions, you should review the introduction for an explanation of the terms and ideas described.

1. Write code to download the text of a book from a url, returning the corpus as a string.
1. Tokenize the corpus into sequences of words and paragraphs.
1. Create two "baseline" langunage models; create text with them using sampling.
1. Compute an N-gram language model
1. Write code to sample from the N-gram language model; create text with them.
1. Evaluate performance (computational) of your language model for different N.
1. Evaluate performance (statistical) of your language model for different N.
1. Create an auto-completion suggestion model.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import project04 as proj

In [3]:
%matplotlib inline

import pandas as pd
import numpy as np
import os
import re
import requests
import time

## Downloading the book(s)

**Question 1**

For this question, you will use `requests` to download and prepare a public domain book from [Project Gutenberg](). Create a function `get_book` that takes in the `url` of a 'Plain Text UTF-8' book and returns a string containing the contents of the book. 

The function should satisfy the following conditions:
* The contents of the book consist of everything between Project Gutenberg's START and END comments.
* The contents *will* include title/author/table of contents.
* You should also transform any Windows new-lines (`\r\n`) with standard new-lines (`\n`).
* You should check the site's `robots.txt` as well and implement a 'pause' in your request in accordance with the website's policy. If the function is called twice in succession, it should not violate the `robots.txt` policy.

*Note:* You are encouraged to find whatever books on the website that interest you to test your code and the language models you develop. The text doesn't even need to be an English-language book.

In [264]:
url = 'http://www.gutenberg.org/files/1661/1661-0.txt'
#url = 'http://www.gutenberg.org/files/57988/57988-0.txt'

In [77]:
r = requests.get(url)
text = r.text

In [78]:
start=text.find('*** START')
end=text.find('*** END')
content=text[start:end]
content=re.sub('\*\*\* START.*\*\*\*','',content)
content=re.sub('\r\n','\n',content)

In [3]:
t=tuple()
len(t)

0

In [24]:
[1,2,3,4,5,6][2:-1]

[3, 4, 5]

In [4]:
# Get the actual book text
url = 'http://www.gutenberg.org/files/74/74-0.txt'
book_string = proj.get_book(url)

## Tokenizing the Corpus

**Question 2**

Now you need to tokenize the corpus in `book_string` (turn the text into a list of tokens -- i.e. words and punctuation). More specifically, create a function `tokenize` that takes in `book_string` and outputs a list of tokens satisfying the following conditions:
* The start of any paragraph should be represented in the list with the single character `\x02` (standing for START).
* The end of any paragraph should be represented in the list with the single character `\x03` (standing for STOP).
* Tokens in the sequence of words are split apart at 'word boundaries' (see the regex lecture).
* Tokens should include *no* whitespace.

For example, the following sentence (the ellipses denote preceding/continuing text):
```
...
My phone's dead.

I didn't get your call.
...
```
should tokenize to:
```
[ ...
"My", "phone", "'", "s", "dead", ".", "\x03", "\x02", "I", "did", "'", "t", "get", "your", "call", "."
... ]
```

*Remark:* Your tokenization function should run quickly; you should avoid loops when possible. Your `tokenize` function should run on the the complete works of Shakespeare (in `data/shakespeare`) in **under 10 seconds** to get full credit. Be sure to only request the book once (using your `get_book` function) and save it to file for testing!

In [5]:
shakespeare_fp = os.path.join('data', 'shakespeare.txt')
shakespeare = open(shakespeare_fp, encoding='utf-8').read()

In [33]:
proj.tokenize('\n\n\n\n\n\n')

['\x02', '\x03', '\x02', '\x03', '\x02', '\x03']

In [32]:
text=shakespeare[:100]
text

'\n\n\n\n\nThe Complete Works of William Shakespeare\n\n\n\nby William Shakespeare\n\n\n\n\n      Contents\n\n\n\n     '

In [28]:
text=re.sub(r'(.)\n', r'\1 \\x03 ', text)
text

'\n\n\n\n\n\nThe Complete Works of William Shakespeare \\x03 \n\n\nby William Shakespeare \\x03 \n\n\n\n      Contents \\x03 \n\n\n     '

In [29]:
text=re.sub(r'\n([^\n]{1})', r' \\x02 \1', text)
text

'\n\n\n\n\n \\x02 The Complete Works of William Shakespeare \\x03 \n\n \\x02 by William Shakespeare \\x03 \n\n\n \\x02       Contents \\x03 \n\n \\x02      '

In [30]:
text = re.sub(r'\\x03', '\x03', text)
text = re.sub(r'\\x02', '\x02', text)
text=re.sub('\n\n', ' \x02 \x03 ', text)
text

' \x02 \x03  \x02 \x03 \n \x02 The Complete Works of William Shakespeare \x03  \x02 \x03  \x02 by William Shakespeare \x03  \x02 \x03 \n \x02       Contents \x03  \x02 \x03  \x02      '

In [31]:
re.findall('(\\b\w+\\b|[^\s])', text)

['\x02',
 '\x03',
 '\x02',
 '\x03',
 '\x02',
 'The',
 'Complete',
 'Works',
 'of',
 'William',
 'Shakespeare',
 '\x03',
 '\x02',
 '\x03',
 '\x02',
 'by',
 'William',
 'Shakespeare',
 '\x03',
 '\x02',
 '\x03',
 '\x02',
 'Contents',
 '\x03',
 '\x02',
 '\x03',
 '\x02']

In [44]:
    text=shakespeare[:50]
    text = re.sub(r'(.)\n\n', r'\1 x03 x02 ', text) # all ending newline
    text = re.sub(r'\n\n([A-Za-z]{1})', r' x03 x02 \1', text)
    text = re.sub('\n\n', ' x03 x02 ', text) # deal with paired left overs
    text = re.sub(r'x02', '\x02', text) 
    text = re.sub(r'x03', '\x03', text)
    pattern = '(\\b\w+\\b|[^\s])'
    s = re.findall(pattern, text)
    
    if s[0] != '\x02':
        s = ['\x02'] + s
    if s[-1] != '\x03':
        s += ['\x03']

In [68]:
text=shakespeare[:60]
text = re.sub(r'(.)\n\n', r'\1 \\x03 \\x02', text)
text = re.sub(r'\n\n(w{1})', r' \\x02 \\x03 \1', text)
text = re.sub('\n\n', '\x03 \x02 ', text)

if len(text) <= 1:
    text = '\x02 ' + text + ' \x03'
if text[0] != '\x02':
    text = '\x02 ' + text
if text[-1] != '\x03':
    text = text + ' \x03'

text = re.sub(r'\\x03', '\x03', text)
text = re.sub(r'\\x02', '\x02', text)
text = re.findall('(\\b\w+\\b|[^\s])', text)
text

['\x02',
 '\x03',
 '\x02',
 '\x03',
 '\x02',
 'The',
 'Complete',
 'Works',
 'of',
 'William',
 'Shakespeare',
 '\x03',
 '\x02',
 '\x03',
 '\x02',
 'by',
 'William',
 '\x03']

In [59]:
proj.tokenize('\x02 \x03')

['\x02', '\x03']

In [120]:
shakespeare_fp = os.path.join('data', 'shakespeare.txt')
shakespeare = open(shakespeare_fp, encoding='utf-8').read()

In [69]:
%timeit proj.tokenize(shakespeare)

758 ms ± 16.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Creating a Language Model

Now that you've learned how to (theoretically) build a language model, you have to implement it in code. Every language model you build will be a class with a few methods in common:

* The `__init__` constructor: when you instantiate an LM object, you will need to pass the 'training corpus' on which your model will be trained (i.e. a list of tokens you created above with `tokenize`). The `train` method will then use that data to create a model which is saved in the `mdl` attribute. This code is given to you.
* The `train` method takes in a list of tokens (e.g. the output of `tokenize`) and outputs a language model. This language model is usually represented as a `Series` (indexed by tokens; values are probabilities that token occurs), or a `DataFrame`.
* The `probability` method takes in a sequence of tokens and returns the probability that this sequence occurs under the language model.
* The `sample` method takes in a number `N > 0` and generates a string made up of `N` tokens using the language model. This method generates language.

You will create three language model classes in the problems below (Uniform; Unigram; N-gram). For each of these, you will implement each of these methods.

## Baseline Models: uniform and unigram LMs

### Uniform probability language model

**Question 3**

A uniform language model is one in which any word is equally likely to appear in any position. The starter code contains a class `UniformLM` which represents a uniform language model. Review the specifics of the class methods above (and in the starter-code) and implement the methods.

In [334]:
class UniformLM(object):
    def __init__(self, tokens):
        self.mdl = self.train(tokens)
    
    def train(self, tokens):
        idx = pd.Series(tokens).unique()
        return pd.Series([1/len(idx)]*len(idx), index=idx)
    
    def probability(self, words):
        try:
            prob=np.prod((pd.Series(words)).apply(lambda x: self.mdl[x]))
            return prob
        except:
            return 0
    
    def sample(self, M):
        return 0

In [343]:
tokens = tuple('one one two three one two four'.split())
unif = UniformLM(tokens)
set(unif.mdl.index) == set('one two three four'.split())

True

In [338]:
unif.mdl.index == set('one two three four'.split())

array([False, False, False, False])

In [328]:
unif = proj.UniformLM(tokens)
unif.sample(100)

'three three one three one two two four three one one one two two one one one three four one three two four three two four one three two four one two three four two two four two four three two three one two one two four two three two four one two three three four one four one two two two four three two three two one two four four three one two four three two one two two one three four two three two one four three four three three four four two three four four three three'

### Unigram language model

**Question 4**

A unigram language model is one in which the likelihood a word appears is proportional to its occurrence in the text. That is, it's just the (unconditional) empirical distribution of words in the corpus. The starter code contains a class `UnigramLM` which represents a unigram language model. Review the specifics of the class methods above (and in the starter-code) and implement the methods.

In [375]:
class UnigramLM(object):
    def __init__(self, tokens):
        self.mdl = self.train(tokens)
    
    def train(self, tokens):
        idx = pd.Series(tokens).unique()
        prob = lambda x: tokens.count(x)/len(tokens)
        return pd.Series(list(map(prob, list(idx))),index=idx)
    
    def probability(self, words):
        try:
            prob=np.prod((pd.Series(words)).apply(lambda x: self.mdl[x]))
            return prob
        except:
            return 0
    
    def sample(self, M):
        sample_lst = np.random.choice(list(self.mdl.index), M, p=list(self.mdl))
        return ' '.join(list(sample_lst))

In [377]:
tokens = tuple('one one two three one two four'.split())
lm=UnigramLM(tokens)
sum(list(lm.mdl))

0.9999999999999998

In [388]:
unigram = proj.UnigramLM(tokens)
sample=unigram.sample(100)
print(unigram.mdl)
(sample.split()).count('four')/100

one      0.428571
two      0.285714
three    0.142857
four     0.142857
dtype: float64


0.13

### Conclusion: Baseline models

You've now trained two baseline language models capable of generating new text from a given training text. Attempt to answer these questions for yourself before you continue.

* Which model do you think is better? Why?
* What are the ways in which both of these models are bad?

## N-gram Model

Now you will build an N-gram language model, which we be a little more involved. The `NGramLM` class contains a few extra methods and attributes beyond those of `UniformLM` and `UnigramLM`:

1. Instantiating `NGramLM` requires both a list of tokens and an `N > 0`, specifying the `N` in N-grams. This parameter is stored in an attribute `.N`.
1. The `NGramLM` class has a method `create_ngrams` that takes in a list of tokens and returns a list of N-grams (an N-gram is a `tuple` of length `N`). This list of N-grams is then passed to the `train` method to train the N-gram model stored in the `mdl` attribute.
1. While the `train` method still creates a language-model (in this case, an N-gram model), this model is most naturally stored as a DataFrame. This DataFrame will have three columns:
    - `'ngram'`: containing the n-grams found in the text.
    - `'n1gram'`: containing the (n-1)-grams upon which the n-grams in `ngram` are built.
    - `'prob'`: containing the probabilities of each n-gram in `ngram`.

### Creating n-grams

**Question 5**

When computing the likelihood that a word occurs in an N-gram model, you compute the probability that word occurs conditional on the (N-1) words that came before it. Therefore, you need to compute a list of n-grams out of your list of tokens (see the introduction for the definition).

*Remark*: What if a word doesn't have (N-1) words preceding it? (e.g. it's near the beginning of the text). How do we compute the likelihood of such a word occurs conditional on the (N-1) words that precede it? To handle this corner case, you should repeat the START (`\x02`) and STOP (`\x03`) tokens (N-1)-times. Thus, if you want to compute the likelihood of the first word $w=w_1$ in the text, you would compute:
$$P(w) = P(w_1) = P(w_1|\texttt{\\x02},\ldots,\texttt{\\x02})$$

Create a method of `NGramLM` called `create_ngrams` that takes in a list of tokens and returns a list of N-grams. The START/STOP tokens in the N-grams should be handled as explained in the remark above.

In [123]:
tokens = tuple('\x02 one two three one four \x03'.split())
tokens

('\x02', 'one', 'two', 'three', 'one', 'four', '\x03')

In [125]:
lm = proj.NGramLM(2,tokens)
lm.mdl

ngram    n1gram  prob
0        (, )      (,)   0.5
1      (, one)      (,)   0.5
2    (one, two)    (one,)   0.5
3  (two, three)    (two,)   1.0
4  (three, one)  (three,)   1.0
5   (one, four)    (one,)   0.5
6     (four, )   (four,)   1.0
7        (, )      (,)   1.0

### Creating the N-gram LM

**Question 6**

Now, you will compute the probabilities that define N-gram language model itself. Recall that the N-gram LM consists of probabilities
$$P(w_1,\ldots,w_N) = P(w_N | w_1,\ldots, w_{N-1}) = \frac{C(w_1, \ldots, w_N)}{C(w_1,\ldots, w_{N-1})}$$

for every n-gram $(w_1, \ldots, w_N)$ that occurs in the text.

Create a method of `NGramLM` called `train` that takes in a list of N-grams and returns a dataframe of conditional probabilities with the following columns:

* `ngram` contains each N-gram that occurs in the text,
* `n1gram` contains the (N-1) tokens that precede the last token in the N-gram occupying the same row.
* `prob` contains the conditional probabilities associated to the N-gram occupying the same row.

In [112]:
tokens=proj.tokenize(shakespeare[:50])
lm=proj.NGramLM(2,tokens)
lm.mdl

ngram          n1gram  prob
0                   (, )            (,)  0.20
1                   (, )            (,)  0.60
2                   (, )            (,)  0.75
3                   (, )            (,)  0.60
4                   (, )            (,)  0.75
5                 (, The)            (,)  0.20
6          (The, Complete)          (The,)  1.00
7        (Complete, Works)     (Complete,)  1.00
8              (Works, of)        (Works,)  1.00
9            (of, William)           (of,)  1.00
10  (William, Shakespeare)      (William,)  1.00
11        (Shakespeare, )  (Shakespeare,)  1.00
12                  (, )            (,)  0.75
13                  (, )            (,)  0.60
14                  (, )            (,)  0.25

In [109]:
tokens = tuple('\x02 one two three one four \x03'.split())
bigrams = proj.NGramLM(2, tokens)
bigrams.mdl

ngram    n1gram  prob
0        (, )      (,)   0.5
1      (, one)      (,)   0.5
2    (one, two)    (one,)   0.5
3  (two, three)    (two,)   1.0
4  (three, one)  (three,)   1.0
5   (one, four)    (one,)   0.5
6     (four, )   (four,)   1.0
7        (, )      (,)   1.0

### Computing probabilities from the N-gram LM

**Question 7**

Create a method of `NGramLM` called `probability` that takes in a 'sentence' (sequence of tokens) and returns the likelihood of the sentence under the language model (see the introduction of the formulas).

*Remark:* What is the probability of a sentence that contains a 'never before seen' word? What about a 'never before seen' (N-1)-gram? Is this reasonable? (Answer: no, it's not reasonable. Fixing this sort of deficiency is a topic in NLP).

In [169]:
tokens = tuple('\x02 one two three one four \x03'.split())
bigrams = proj.NGramLM(2, tokens)
words='one two five'.split()
n=2
df=bigrams.mdl
tuple(words)

('one', 'two', 'five')

In [170]:
app=lambda x: tuple([words[i] for i in range(x, x+n)])
tokens=list(map(app, list(range(len(words)-n+1))))
#find_prob=lambda x: df[df['ngram']==tokens[x]]['prob'].iloc[0]
#np.prod(list(map(find_prob, list(range(len(tokens))))))

In [171]:
tokens[0]

[('one', 'two'), ('two', 'five')]

### Sampling from the N-gram LM

**Question 8**

As you saw creating the baseline models, sampling from your trained LM provides a way to generate new language. You will code up a `sample` method for `NGramLM` and observe how much better (or worse) the generated language appears when compared to your baseline models.

Create a method of `NGramLM` called `sample` that takes in a number `M > 0` and generates a string of M tokens using the language model (not counting the initial START token(s)). It should sample a first token of the form $(\texttt{\\x02},\ldots,\texttt{\\x02}, w_1)$ from the probabilities in `self.mdl` and continue picking words conditional on the previous choice. Helper functions and recursion will be very helpful.

*Remark:* If you find, at any point, your LM has no words to sample, then you should return the STOP token (`\x03`) and continue until you reach the required length.

*(Fun) Remark:* Try generating text using different values of `N`. What are the differences (qualitatively)? Don't be alarmed if your samples generate strange quasi-coherent sentences; it's just randomness!

*(Fun) Remark:* Try sampling from an `N`-gram model built on books written in different languages (It doesn't even need to be written using the Latin alphabet!)

In [358]:
tokens = tuple('\x02 one two three one four \x03'.split())
bigrams = proj.NGramLM(2,tokens)
n=2
bigrams.mdl

ngram    n1gram  prob
0        (, )      (,)   0.5
1      (, one)      (,)   0.5
2    (one, two)    (one,)   0.5
3  (two, three)    (two,)   1.0
4  (three, one)  (three,)   1.0
5   (one, four)    (one,)   0.5
6     (four, )   (four,)   1.0
7        (, )      (,)   1.0

In [372]:
df=bigrams.mdl
first = ('\x02', )*(n-1)
df_prob = df[df['n1gram']==first]
s=' '.join(first)+' '+(np.random.choice(df_prob['ngram'],1,p=df_prob['prob']))[0][-1]
s

'\x02 \x02'

In [360]:
def generate(sentence, count):
    print(sentence)
    if count == 0:
        return sentence
    
    token = tuple(sentence.split()[-(n-1):])
    df_prob = df[df['n1gram']==token]
    sentence=sentence+' '+(np.random.choice(df_prob['ngram'],1,p=df_prob['prob']))[0][-1]
    count -=1
    return generate(sentence, count)

In [375]:
generate(s,3-1)

 
  
   


'\x02 \x02 \x02 \x02'

In [409]:
ngram = proj.NGramLM(2, tokens)

In [133]:
tokens = tuple('\x02 one two three one four \x03'.split())
ngrams = proj.NGramLM(3, tokens)
ngrams.sample(10)

'\x02 \x02 \x02 one two three one four \x03 \x03 \x03 \x03'

In [156]:
tokens = tuple("\x02 My phone's dead.\x03 \x02 I didn't get your call.\x03".split())
ngrams = proj.NGramLM(4,tokens)
ngrams.sample(15)

"\x02 \x02 \x02 \x02 My phone's dead.\x03 \x02 I didn't get your call.\x03 \x03 \x03 \x03 \x03 \x03"

In [120]:
        n=4
        df = ngrams.mdl
        first = ('\x02',) * (n - 1)
        df_prob = df[df['n1gram'] == first]
        s = ' '.join(first) + ' ' + (np.random.choice(df_prob['ngram'], 1, p=df_prob['prob']))[0][-1]
        s

'\x02 \x02 \x02 My'

In [123]:
        def generate(sentence, count):
            if count == 0:
                return sentence

            token = tuple(sentence.split()[-(n - 1):])
            df_prob = df[df['n1gram'] == token]
            sentence = sentence + ' ' + (np.random.choice(df_prob['ngram'], 1, p=df_prob['prob']))[0][-1]
            count -= 1
            return generate(sentence, count)

In [124]:
generate(s,4)

"\x02 \x02 \x02 My phone's dead.\x03 \x02 I"

### Predicting the next word in a sentence

**Question 9**

Next, you will build a predictor that predicts the most likely word to follow a given sentence. The predictions will be the *maximum likelihood estimate* given by your N-gram model. Recall that your N-gram model contains the probabilities that a token follows an (n-1)-gram; your predictor will pick the token with the highest probability of occurring. 

Create a function `predict_next` that takes in an instantiated `NGramLM` object and a list of tokens, and predicts the *most likely token* to follow the list of tokens in the input (according to `NGramLM`).

*Remark:* For which N does this predictor work best? (Determine this by getting a feel for the output, we won't precisely answer this question) .

*Remark:* What would this function look like if it were to take in `UniformLM` or `UnigramLM`?

In [82]:
tokens = tuple('\x02 one two three one four \x03'.split())
bigrams = proj.NGramLM(3, tokens)
df=bigrams.mdl
df

ngram        n1gram  prob
0           (, , )        (, )   0.5
1         (, , one)        (, )   0.5
2       (, one, two)      (, one)   1.0
3   (one, two, three)    (one, two)   1.0
4   (two, three, one)  (two, three)   1.0
5  (three, one, four)  (three, one)   1.0
6      (one, four, )   (one, four)   1.0
7        (four, , )     (four, )   1.0
8           (, , )        (, )   1.0

In [94]:
if not isinstance(tokens, tuple):
    tokens = tuple(tokens)
n1gram = tokens[-(n-1):]
sort=df[df['n1gram']==n1gram].sort_values(by='prob')
if sort.shape[0] == 0:
    print(True)
else:
    print((sort['ngram'].iloc[0])[-1])
df

True


ngram        n1gram  prob
0           (, , )        (, )   0.5
1         (, , one)        (, )   0.5
2       (, one, two)      (, one)   1.0
3   (one, two, three)    (one, two)   1.0
4   (two, three, one)  (two, three)   1.0
5  (three, one, four)  (three, one)   1.0
6      (one, four, )   (one, four)   1.0
7        (four, , )     (four, )   1.0
8           (, , )        (, )   1.0

In [72]:
proj.predict_next(proj.NGramLM(2,tokens), ('one', 'two', 'three'))

'one'

In [470]:
proj.NGramLM(3,tokens).mdl

ngram        n1gram  prob
0           (, , )        (, )   0.5
1         (, , one)        (, )   0.5
2       (, one, two)      (, one)   1.0
3   (one, two, three)    (one, two)   1.0
4   (two, three, one)  (two, three)   1.0
5  (three, one, four)  (three, one)   1.0
6      (one, four, )   (one, four)   1.0
7        (four, , )     (four, )   1.0
8           (, , )        (, )   1.0

## Model Performance Evaluation (Extra Credit)

**Question 10**

This question attempts to answer the question "which choice of `N` is best when creating an N-gram model?"

As you likely noticed when generating text using your N-gram models, the sentences appear to become more coherent as N increases. The naive conclusion would be to make N very large. However, such a choice has a downside that you will investigate.

One concern is that as N gets larger, the distribution of N-grams get more *sparse* -- i.e. the collection of n-grams all have counts close to 1. This makes the language model more likely to merely generate text that was already present in the original text, as opposed to generating *new* language (why?). 

In this question, you will quantify this observation: given an N-gram model and a sample of length M, what percentage of (N+1)-grams in the sample are present in the text that was used to create the model?

Create a function `evaluate_models` which takes in a list of tokens and:
1. Generates N-gram models for `N=2,3,4,5,6,7,8` from the given list of tokens,
2. Creates samples of length 1000 from each model (remove START/STOP tokens from the samples),
3. Calculates the proportion of (N+1)-grams in each sample that was already present in the original list of tokens.

The function should return the proportions in a list of length 6.